### Importação dos pacotes

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

### Pré-processamento nos dados

In [4]:
# lendo os dados de um arquivo csv
dataframe = pd.read_csv('/content/drive/MyDrive/vinbigdata/train.csv')
# criando uma coluna com os caminhos relativos as imagens
dataframe['image_path'] = '/content/drive/MyDrive/vinbigdata/train/' + dataframe.image_id + '.jpg'

In [5]:
print('total de imagens disponíveis:', str(len(set(dataframe['image_path']))))

total de imagens disponíveis: 15000


In [6]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Aortic enlargement     7162
Cardiomegaly           5427
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Other lesion           2203
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [7]:
# removendo os casos não relativos a distúrbios pulmonares
dataframe = dataframe[dataframe.class_name != 'Aortic enlargement']
dataframe = dataframe[dataframe.class_name != 'Cardiomegaly']
dataframe = dataframe[dataframe.class_name != 'Other lesion']
dataframe = dataframe[dataframe.class_name != 'Consolidation']

In [8]:
# separando os casos rotulados como normais e anormais
normal_cases = dataframe[(dataframe.class_id == 14) & (dataframe.class_name == 'No finding')]
abnormal_cases = dataframe[(dataframe.class_id != 14) & (dataframe.class_name != 'No finding')]

print('total de dados após a filtração:', str(len(set(normal_cases['image_path'])) + len(set(abnormal_cases['image_path']))))

total de dados após a filtração: 13948


In [16]:
# removendo as imagens repetidas
normal_data = normal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )
abnormal_data = abnormal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )

# criando dataframes especifos com caminhos para as imagens e rótulos
normal_data['target'] = 'normal'
abnormal_data['target'] = 'abnormal'

In [17]:
print('quantidade de dados rotulados como normais:', len(normal_data))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 10606
quantidade de dados rotulados como anormais: 3342


In [18]:
# removendo 69% dos casos normais para balancear os dados
normal, _ = train_test_split(normal_data, test_size = 0.69, random_state = 42)

In [19]:
print('quantidade de dados rotulados como normais:', len(normal))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 3287
quantidade de dados rotulados como anormais: 3342


In [20]:
# concatenando os dataframes de casos normais e anormais
full_data = pd.concat([normal, abnormal_data])

In [ ]:
# misturando todos os dados do dataframe e reiniciando os valores dos índices 
full_data = full_data.sample(frac = 1, axis = 0, random_state = 42).reset_index(drop=True)

In [34]:
full_data = full_data[full_data.class_name != 'No finding']
full_data = full_data[full_data.class_name != 'Atelectasis']
full_data = full_data[full_data.class_name != 'Pneumothorax']

In [35]:
full_data['class_name'].value_counts()

Pleural thickening    908
Pulmonary fibrosis    746
Lung Opacity          448
Nodule/Mass           347
Pleural effusion      332
Infiltration          169
Calcification         167
ILD                   152
Name: class_name, dtype: int64

In [36]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['class_name'],
                                     test_size = 0.2, random_state = 42)

In [37]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['class_name'],
                                           test_size = 0.2, random_state = 42)

In [38]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['image_path']))
print('quantidade de rótulos de treinamento:', len(train_df['class_name']))
print('quantidade de imagens de teste:', len(test_df['image_path']))
print('quantidade de rótulos de teste:', len(test_df['class_name']))
print('quantidade de imagens de validação:', len(validation_df['image_path']))
print('quantidade de rótulos de validação:', len(validation_df['class_name']))

quantidade de imagens de treinamento: 2092
quantidade de rótulos de treinamento: 2092
quantidade de imagens de teste: 654
quantidade de rótulos de teste: 654
quantidade de imagens de validação: 523
quantidade de rótulos de validação: 523


In [39]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255.,
                                     rotation_range = 10, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))
# criando o gerador de imagens de validação 
valid_generator = image_generator.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(rescale = 1./255.)

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'image_path',
                                                  y_col = 'class_name',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 2092 validated image filenames belonging to 8 classes.
Found 523 validated image filenames belonging to 8 classes.
Found 654 validated image filenames belonging to 8 classes.


### Preparando a rede neural convolucional

In [40]:
# baixando os pesos treinados da rede inception
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-05-06 20:11:26--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.1.128, 108.177.121.128, 142.250.103.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.1.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   129MB/s    in 0.6s    

2021-05-06 20:11:27 (129 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [41]:
# referenciando o local em que os pesos estão armazenados
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# carregando a arquitetura inception pré-treinada
pre_trained_model = InceptionV3(input_shape = (256, 256, 3), 
                                include_top = False, 
                                weights = None)

# carregando os pesos treinados com outros dados 
pre_trained_model.load_weights(local_weights_file)

# definindo as flags iniciais  
#pre_trained_model.trainable = True
#set_trainable = False

# para a arquitetura inception, a rede será retreinada a partir da camada 'mixed8'
#for layer in pre_trained_model.layers:
#    if layer.name == 'mixed8':
#        set_trainable = True
#    if set_trainable:
#        layer.trainable = True
#    else:
#       layer.trainable = False

# visualizando a arquitetura definida
#pre_trained_model.summary()

# obtendo a última camada como sendo a nomeada por 'mixed7'
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [42]:
# definindo uma camada de achatamento
x = layers.Flatten()(last_output)
# conecatando a rede uma camada com 1024 neurônios e função de ativação relu
x = layers.Dense(units = 512, activation = 'relu')(x)     
# conecatando a rede uma camada com 128 neurônios e função de ativação relu
x = layers.Dense(units = 256, activation = 'relu')(x) 
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x)                  
# adicionando uma camada de saída com um neurônio e uma função de ativação sigmoide
x = layers.Dense  (units = 8, activation = 'softmax')(x)           

# conecatando as camadas definidas acima com a arquitetura inception
model = Model(pre_trained_model.input, x) 

# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [43]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [44]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [45]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 2092 // 32, 
                              validation_data = valid_generator, validation_steps = 523 // 32,
                              callbacks = callbacks, epochs = 10)

Epoch 1/10
65/65 [==============================] - 1832s 28s/step - loss: 0.4242 - acc: 0.2101 - val_loss: 0.3572 - val_acc: 0.2754

Epoch 00001: val_acc improved from -inf to 0.27539, saving model to transferlearning_weights.hdf5
Epoch 2/10
65/65 [==============================] - 1364s 21s/step - loss: 0.3541 - acc: 0.2679 - val_loss: 0.3440 - val_acc: 0.3105

Epoch 00002: val_acc improved from 0.27539 to 0.31055, saving model to transferlearning_weights.hdf5
Epoch 3/10
65/65 [==============================] - 1390s 21s/step - loss: 0.3319 - acc: 0.3508 - val_loss: 0.3411 - val_acc: 0.3223

Epoch 00003: val_acc improved from 0.31055 to 0.32227, saving model to transferlearning_weights.hdf5
Epoch 4/10
65/65 [==============================] - 1382s 21s/step - loss: 0.3096 - acc: 0.3959 - val_loss: 0.3502 - val_acc: 0.3047

Epoch 00004: val_acc did not improve from 0.32227
Epoch 5/10
65/65 [==============================] - 1376s 21s/step - loss: 0.2844 - acc: 0.4708 - val_loss: 0.3576

In [46]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 2092 // 32, 
                              validation_data = valid_generator, validation_steps = 523 // 32,
                              callbacks = callbacks, epochs = 10)

Epoch 1/10
65/65 [==============================] - 1411s 22s/step - loss: 0.0958 - acc: 0.8801 - val_loss: 0.7163 - val_acc: 0.3379

Epoch 00001: val_acc did not improve from 0.35156
Epoch 2/10
65/65 [==============================] - 1446s 22s/step - loss: 0.0787 - acc: 0.9005 - val_loss: 0.6395 - val_acc: 0.3164

Epoch 00002: val_acc did not improve from 0.35156
Epoch 3/10
65/65 [==============================] - 1420s 22s/step - loss: 0.0652 - acc: 0.9277 - val_loss: 0.7167 - val_acc: 0.2676

Epoch 00003: val_acc did not improve from 0.35156
Epoch 4/10
65/65 [==============================] - 1394s 21s/step - loss: 0.0630 - acc: 0.9233 - val_loss: 0.7035 - val_acc: 0.3066

Epoch 00004: val_acc did not improve from 0.35156
Epoch 5/10
65/65 [==============================] - 1403s 22s/step - loss: 0.0476 - acc: 0.9515 - val_loss: 1.0127 - val_acc: 0.3164

Epoch 00005: val_acc did not improve from 0.35156
Epoch 6/10
65/65 [==============================] - 1425s 22s/step - loss: 0.043

KeyboardInterrupt: ignored

In [47]:
# carregando o melhor peso obtido para o modelo
best_model = model
best_model.load_weights('/content/transferlearning_weights.hdf5')

In [48]:
# salvando os dois modelos obtidos durante o treinamento
model.save('model1')
best_model.save('model2')

INFO:tensorflow:Assets written to: model1/assets
INFO:tensorflow:Assets written to: model2/assets


In [49]:
best_model.evaluate(test_generator)

21/21 [==============================] - 387s 19s/step - loss: 0.5634 - acc: 0.3058


[0.5634042024612427, 0.3058103919029236]

In [51]:
# carregando os dados de teste
for i in range(0, 21):
  (x1, y1) = test_generator[i]
  if i == 0:
    x, y = x1, y1
  else:
    x = np.concatenate((x, x1))
    y = np.concatenate((y, y1))

In [56]:
# realizando a predição para os dados de teste
predict = model.predict(x)

count_global = 0
for predicts in predict:
    count = 0
    aux = np.zeros((8,))
    for values in predicts:
        if values >= 0.45:
            aux[count] = 1.
        else:
            aux[count] = 0.
        count += 1
    predict[count_global] = aux
    count_global += 1

In [57]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
print('Matriz de Confusão:\n', confusion_matrix(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Acurácia:', accuracy_score(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Precisão', precision_score(y.argmax(axis = 1), predict.argmax(axis = 1), average = 'weighted'))
print('Sensibilidade:', recall_score(y.argmax(axis = 1), predict.argmax(axis = 1), average = 'weighted')) 
print('F1_Score:', f1_score(y.argmax(axis = 1), predict.argmax(axis = 1), average = 'weighted'))

Matriz de Confusão:
 [[  5   2   0   2   0   0  19   5]
 [  4   5   0   3   0   0  15   3]
 [  5   1   1   2   0   2  14   9]
 [  4   0   0  11   4  12  45  14]
 [  7   1   0  12   3   2  28  17]
 [  5   0   0   6   0  14  32   9]
 [ 11   3   0  12   1   9 120  26]
 [  8   1   0   6   0  14  80  40]]
Acurácia: 0.30428134556574926
Precisão 0.3382998119462324
Sensibilidade: 0.30428134556574926
F1_Score: 0.2646428256069972
